### Improvised Previous Models and Better RMSE score 
- Shuffle= False in model.fit()  #Keras shuffles the training dataset before each training epoch. To ensure the training data patterns remain sequential, we can disable this shuffling.
- increased epochs and batch size
- [Scaling Test Data before Predictions](https://stackoverflow.com/questions/50565937/how-to-normalize-the-train-and-test-data-using-minmaxscaler-sklearn/50567308)

In [1]:
import pandas as pd
from sklearn .preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
from math import sqrt
from numpy import split
from numpy import array
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [3]:
from matplotlib import pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
pd.options.display.float_format = '{:,.10f}'.format

In [4]:
data  = pd.read_excel('emotion.xls').drop('Date', axis=1 ).set_index("day")

In [5]:
data = data.astype('float32')

In [6]:
data_vol =  data.iloc[ : , 0:13].drop(['% change_nifty', '% change_sensex'], axis= 1)

In [7]:
data_vol.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3168 entries, 2007-04-19 to 2020-03-16
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Anger                   3168 non-null   float32
 1   Anticipation            3168 non-null   float32
 2   Disgust                 3168 non-null   float32
 3   Fear                    3168 non-null   float32
 4   Joy                     3168 non-null   float32
 5   Sadness                 3168 non-null   float32
 6   Surprise                3168 non-null   float32
 7   Trust                   3168 non-null   float32
 8   Close_nifty             3168 non-null   float32
 9   Close_sensex            3168 non-null   float32
 10  Conditional Volatility  3168 non-null   float32
dtypes: float32(11)
memory usage: 160.9 KB


In [8]:
data_vol.head()

,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Close_nifty,Close_sensex,Conditional Volatility
day,,,,,,,,,,,
2007-04-19,0.5000000000,2.2100000381,0.0000000000,1.5160000324,0.8130000234,0.4309999943,0.0000000000,1.0470000505,"3,997.6499023438","13,619.7001953125",1.9240180254
2007-04-23,0.4850000143,1.3819999695,0.0000000000,0.5780000091,1.0479999781,2.6700000763,0.0000000000,1.1720000505,"4,085.1000976562","13,928.3300781250",1.9290779829
2007-04-30,0.0000000000,2.7019999027,0.0000000000,0.2969999909,1.6189999580,0.0000000000,0.0000000000,1.6410000324,"4,087.8999023438","13,872.3701171875",2.1650118828
2007-05-07,0.5310000181,3.1619999409,0.0000000000,0.2969999909,2.6429998875,0.2029999942,0.3670000136,3.5710000992,"4,111.1499023438","13,879.2500000000",2.0097639561
2007-05-08,0.0000000000,1.0069999695,0.0000000000,0.0000000000,0.8930000067,0.0000000000,0.0000000000,1.0470000505,"4,077.0000000000","13,765.4599609375",1.8497519493


## Train the Model

In [121]:
# def split_dataset(data):
# 	# split into  months
#     train, test = data[0:3138], data[3138:3168]
# 	# restructure into windows of monthly data
#     train = array(split(train, len(train)/30))
#     test = array(split(test, len(test)/30))
#     return train, test

In [9]:
yscaler = MinMaxScaler()
df = pd.DataFrame(data_vol.iloc[18:3138, 10])
df = pd.DataFrame(yscaler.fit_transform(df), columns=df.columns, index=df.index)

In [10]:
scaler = MinMaxScaler()
train, test = data_vol[18:3138], data_vol[3138:3168]
train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)

In [16]:
test = pd.DataFrame(scaler.fit_transform(test), columns=test.columns, index=test.index)

In [11]:
train.describe()

,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Close_nifty,Close_sensex,Conditional Volatility
count,"3,120.0000000000","3,120.0000000000","3,120.0000000000","3,120.0000000000","3,120.0000000000","3,120.0000000000","3,120.0000000000","3,120.0000000000","3,120.0000000000","3,120.0000000000","3,120.0000000000"
mean,0.0667189583,0.1310477257,0.0656884834,0.0793018043,0.1180894151,0.1094094813,0.0957049057,0.0974604413,0.4660172760,0.4551837146,0.0627599508
std,0.1343169659,0.1967794001,0.1465145200,0.1510540098,0.1811063737,0.1672122926,0.1806580275,0.1728436798,0.2494771630,0.2389431000,0.1095687598
min,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
25%,0.0102568883,0.0386259221,0.0000000000,0.0146211691,0.0332342461,0.0263262000,0.0130911404,0.0220476161,0.2735500485,0.2722832114,0.0140771582
50%,0.0205137767,0.0591193344,0.0147001697,0.0273765000,0.0526462533,0.0504153557,0.0318445377,0.0353673287,0.3654592931,0.3632382154,0.0270586554
75%,0.0390222305,0.1055044997,0.0361902015,0.0472020544,0.0934748799,0.0913505517,0.0642666277,0.0630328879,0.6322714686,0.6030966938,0.0569391027
max,0.9999999404,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000


In [17]:
test.describe()

,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Close_nifty,Close_sensex,Conditional Volatility
count,30.0000000000,30.0000000000,30.0000000000,30.0000000000,30.0000000000,30.0000000000,30.0000000000,30.0000000000,30.0000000000,30.0000000000,30.0000000000
mean,0.5515988469,0.5071460009,0.4846850038,0.5973227024,0.5359319448,0.4792217612,0.4619318247,0.3523563147,0.7586560845,0.7619783282,0.1437718421
std,0.2692614794,0.2503173649,0.2438482940,0.2421970218,0.2378204912,0.2652696371,0.2123727500,0.2050618678,0.2697877586,0.2673042417,0.2423311025
min,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
25%,0.3426034003,0.3419924080,0.2841547430,0.4213398844,0.3571308702,0.2642595172,0.3399322331,0.2331271619,0.6713703275,0.6815431118,0.0386144165
50%,0.5611848235,0.5451586843,0.5261563659,0.6132943630,0.5389156938,0.4636054635,0.4811083972,0.3379986882,0.8507556915,0.8544769287,0.0506458730
75%,0.7835264802,0.6754058301,0.6659611762,0.7706734836,0.7058766186,0.6165616512,0.5351662338,0.4581854045,0.9620062113,0.9583874941,0.1325604618
max,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,0.9999999404,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000


In [18]:
train = array(split(train.values, len(train)/30))

In [19]:
test = array(split(test.values, len(test)/30))

In [13]:
# scaler = MinMaxScaler()
# for col in data_vol.columns :
# # for i in range (0,train.shape[2]) :
#     data_vol[[col]] = scaler.fit_transform(data_vol[[col]])
# #     train[:,:, i] = scaler.fit_transform(train[:,:,i])

In [14]:
def to_supervised(train, n_input, n_out=30):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	for _ in range(len(data)):
		in_end = in_start + n_input
		out_end = in_end + n_out
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 10])
		in_start += 1
	return array(X), array(y)

In [15]:
def build_model(train, n_input):
	train_x, train_y = to_supervised(train, n_input)
	verbose, epochs, batch_size = 1, 300, 100
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
	print(train_x.shape)
	print(train_y.shape)
    
	# define model
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(RepeatVector(n_outputs))
	model.add(LSTM(200, activation='relu', return_sequences=True))
	model.add(TimeDistributed(Dense(100, activation='relu')))
	model.add(TimeDistributed(Dense(1)))
	model.compile(loss='mse', optimizer='adam')
	model.summary()
    
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose, shuffle= False )
	return model

In [16]:
n_input = 30 
model = build_model(train, n_input) 
model.save( 'model5-copy2' + '.h5')
print("model saved")

(3061, 30, 11)
(3061, 30, 1)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               169600    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 30, 200)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 200)           320800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 100)           20100     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 1)             101       
Total params: 510,601
Trainable params: 510,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
3061/3061 [==============================] - 22s 7ms/step - loss: 0.0147


### Prediction and Evaluation

In [11]:
from numpy import loadtxt
from keras.models import load_model  
from keras.utils.vis_utils import plot_model

In [12]:
model = load_model("model5-copy2.h5")

In [13]:
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=1)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

In [21]:
def evaluate_forecasts_new(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
        
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

In [22]:
def evaluate_model_new(train, test, n_input):
    
	# history is a list of weekly data
	history = [x for x in train]
    
	# walk-forward validation 
	predictions = list()
    
	for i in range(len(test)):
		yhat_sequence = forecast(model, history, n_input)
# 		print(type(yhat_sequence))
# 		print(yhat_sequence.shape) 
# 		y1 = scaler.inverse_transform(yhat_sequence)
# 		print(type(y1))
# 		print(y1.shape)    
		predictions.append(yhat_sequence)
		history.append(test[i, :])
        
	# evaluate predictions days for each week
	predictions = array(predictions)
# 	print(predictions.shape)  
# 	predictions = scaler.inverse_transform(predictions[0,:,:])
	return predictions

In [23]:
preds = evaluate_model_new(train, test, 30)

1/1 [==============================] - 1s 717ms/step


In [33]:
print(test[:,10])

[[0.48979443 0.5681981  0.4947866  0.4829613  0.44494575 0.4523285
  0.44238916 0.27872235 0.97078705 0.96971655 0.03399805]]


In [24]:
print(preds)

[[[0.01868879]
  [0.01977039]
  [0.01935527]
  [0.01970544]
  [0.02059191]
  [0.02178053]
  [0.02307583]
  [0.02329719]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]
  [0.02297284]]]


In [26]:
score, scores = evaluate_forecasts_new(test[:, :, 8], preds)

In [27]:
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

In [28]:
summarize_scores('test', score, scores)

test: [0.783] 0.8, 0.8, 0.9, 0.9, 1.0, 0.9, 0.9, 0.9, 1.0, 1.0, 0.9, 0.9, 0.9, 1.0, 0.9, 0.9, 0.8, 0.8, 0.8, 0.6, 0.6, 0.7, 0.7, 0.7, 0.6, 0.4, 0.4, 0.1, 0.2, 0.0
